# Visualization Design

By: *Tyler Biggs*

---

**Overview**

This notebook will go over the design of the vizualizations. It should also serve as a reference for future custom visualizations.

In [2]:
%load_ext autoreload
%autoreload 2
from pprint import pprint

In [3]:
import pandas as pd
import numpy as np
import re
import collections
import itertools
import functools

import bokeh as bk
import bokeh.io
import bokeh.models
import bokeh.layouts
import bokeh.plotting
bokeh.io.output_notebook()

# import holoviews as hv
# hv.extension('bokeh')

Loading BokehJS ...

In [4]:
# Path hack to allow imports from the parent directory.
import sys, os
sys.path.insert(0, os.path.abspath('../../'))

In [5]:
from isadream.isadream import modelUtils
from isadream.isadream import io

---

## Dataflow

The data is transfered from the Drupal server as a `.json` file. Those files placed into a directory as the user requests them. That is, all the datasets that a user selects for any given visualziation are placed in a directory. These files are condensed into four dataframes per `.json` file.

In [6]:
# A demo json file is provided.
nmr_json_demo = modelUtils.SIPOS_DEMO
demo_base_path = modelUtils.BASE_PATH
print(nmr_json_demo, '\n', demo_base_path)

/home/tylerbiggs/git/isadream/isadream/demo_data/demo_json/sipos_2006_talanta_nmr_figs.json 
 /home/tylerbiggs/git/isadream/isadream/demo_data


---

In [7]:
demo_json = io.read_idream_json(nmr_json_demo)
node = io.parse_json(demo_json)

test_groups = (
    ('27 Al ppm', 'ppm', ("Al",)),
    ('Total Aluminate Concentration', 'Molar', ("Al", )),
    ('Counter Ion Concentration', 'Molar', ("Na+", "Li+", "Cs+", "K+", )),
    ('Counter Ion', 'Species', ("Na+", "Li+", "Cs+", "K+", )),
    ('Base Concentration', 'Molar', ("OH-", )),
)

node_assays = [a.build_column_data_dicts(test_groups) for a in node.assays]

dfs = [pd.DataFrame(cds) for cds, _ in node_assays]
df = pd.concat(dfs)
df.T

,0,1,2,3,4,0,1,2,3,4,...,7,0,1,2,3,4,5,6,7,8
parent_node,2129d998-1f62-46e6-90e8-1db86de60c33,2129d998-1f62-46e6-90e8-1db86de60c33,2129d998-1f62-46e6-90e8-1db86de60c33,2129d998-1f62-46e6-90e8-1db86de60c33,2129d998-1f62-46e6-90e8-1db86de60c33,bdae4414-99ec-4ceb-a12a-fc8908de05fb,bdae4414-99ec-4ceb-a12a-fc8908de05fb,bdae4414-99ec-4ceb-a12a-fc8908de05fb,bdae4414-99ec-4ceb-a12a-fc8908de05fb,bdae4414-99ec-4ceb-a12a-fc8908de05fb,...,bdae4414-99ec-4ceb-a12a-fc8908de05fb,fb05fa29-fbb1-45d4-bd54-a1c705bcbe7c,fb05fa29-fbb1-45d4-bd54-a1c705bcbe7c,fb05fa29-fbb1-45d4-bd54-a1c705bcbe7c,fb05fa29-fbb1-45d4-bd54-a1c705bcbe7c,fb05fa29-fbb1-45d4-bd54-a1c705bcbe7c,fb05fa29-fbb1-45d4-bd54-a1c705bcbe7c,fb05fa29-fbb1-45d4-bd54-a1c705bcbe7c,fb05fa29-fbb1-45d4-bd54-a1c705bcbe7c,fb05fa29-fbb1-45d4-bd54-a1c705bcbe7c
assay_node,c6561c3c-d15b-4825-b051-e90406930360,c6561c3c-d15b-4825-b051-e90406930360,c6561c3c-d15b-4825-b051-e90406930360,c6561c3c-d15b-4825-b051-e90406930360,c6561c3c-d15b-4825-b051-e90406930360,a8f162c8-ffb3-4e08-97b5-5827e7d942e4,a8f162c8-ffb3-4e08-97b5-5827e7d942e4,a8f162c8-ffb3-4e08-97b5-5827e7d942e4,a8f162c8-ffb3-4e08-97b5-5827e7d942e4,a8f162c8-ffb3-4e08-97b5-5827e7d942e4,...,a8f162c8-ffb3-4e08-97b5-5827e7d942e4,fa28a85e-a75a-4b5e-a2cd-465c6a76a01a,fa28a85e-a75a-4b5e-a2cd-465c6a76a01a,fa28a85e-a75a-4b5e-a2cd-465c6a76a01a,fa28a85e-a75a-4b5e-a2cd-465c6a76a01a,fa28a85e-a75a-4b5e-a2cd-465c6a76a01a,fa28a85e-a75a-4b5e-a2cd-465c6a76a01a,fa28a85e-a75a-4b5e-a2cd-465c6a76a01a,fa28a85e-a75a-4b5e-a2cd-465c6a76a01a,fa28a85e-a75a-4b5e-a2cd-465c6a76a01a
sample_node,df618af3-169c-4ac8-8d12-70d0af8a701d,df618af3-169c-4ac8-8d12-70d0af8a701d,df618af3-169c-4ac8-8d12-70d0af8a701d,df618af3-169c-4ac8-8d12-70d0af8a701d,df618af3-169c-4ac8-8d12-70d0af8a701d,de0d36b3-785e-43fa-b383-0b7b9eb9768c,de0d36b3-785e-43fa-b383-0b7b9eb9768c,de0d36b3-785e-43fa-b383-0b7b9eb9768c,de0d36b3-785e-43fa-b383-0b7b9eb9768c,de0d36b3-785e-43fa-b383-0b7b9eb9768c,...,de0d36b3-785e-43fa-b383-0b7b9eb9768c,84f30aa9-c23a-4405-950c-c896cc83871b,84f30aa9-c23a-4405-950c-c896cc83871b,84f30aa9-c23a-4405-950c-c896cc83871b,84f30aa9-c23a-4405-950c-c896cc83871b,84f30aa9-c23a-4405-950c-c896cc83871b,84f30aa9-c23a-4405-950c-c896cc83871b,84f30aa9-c23a-4405-950c-c896cc83871b,84f30aa9-c23a-4405-950c-c896cc83871b,84f30aa9-c23a-4405-950c-c896cc83871b
27 Al ppm,79.9,79.84,79.72,79.66,79.66,79.92,79.84,79.66,79.54,79.36,...,78.47,79.54,79.98,79.13,78.45,77.67,76.47,74.53,73.14,71.32
Total Aluminate Concentration,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.005,...,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.005
Counter Ion Concentration,2.93,4.92,6.85,9.13,10.71,0.66,1.1,1.64,2.14,2.59,...,4.11,4.98,0.93,6.97,8.96,10.95,12.99,14.92,16.97,18.92
Counter Ion,K+,K+,K+,K+,K+,Li+,Li+,Li+,Li+,Li+,...,Li+,Na+,Na+,Na+,Na+,Na+,Na+,Na+,Na+,Na+
Base Concentration,2.93,4.92,6.85,9.13,10.71,0.66,1.1,1.64,2.14,2.59,...,4.11,4.98,0.93,6.97,8.96,10.95,12.99,14.92,16.97,18.92


In [14]:
mddfs = [pd.DataFrame(md) for _, md in node_assays]
mdf = pd.concat(mddfs, sort=False, axis=1)
mdf

,e30d62f1-a7fa-4f5e-b6db-8d586e45b0bb,1f91387d-becf-4d8d-9b11-17abdfb1340c,ddc51c79-5f81-4d82-a36d-0186fe77f818,2b3330d4-3479-4fc4-87e3-1e524b0bc671,55de257d-0f71-4941-b5d5-2f95cad9cc82,8ff56fb7-1f17-4f30-9621-fe8137314d8b,d73d22f7-79a8-41d1-9d15-0e4b23679d5d,f3a82cea-3e58-41d7-a7f2-ce57e5ba562b,3bc0c656-7d5c-4729-a80b-153165b5c2e4,bb5e7a02-fadf-4dbc-97bd-09fc974e5541,...,5c16a2e1-115d-4d3c-a334-b3d5aff0468f,b3cbab39-ca39-4480-b7d3-9b5f70ea1d3f,c5ffaf2c-10fe-40e9-888e-2958af973374,81806080-1b04-4d79-b59b-65892c5c6f5c,bcc00694-8a3c-4335-9bc5-fd73c6cd4d59,a98001af-4d92-4247-8779-ec40ab63541e,2ef26e5e-9670-43b8-ac35-d5e8e734d5fb,ba0478bb-59c5-4c48-b591-cdeb97872e21,37e35404-ac11-4446-a8bf-b1d989fe6a62,4e97ea98-ded0-4bf8-b8c7-ed48baf68dc8
$id,https://lampdev02.pnl.gov/bigg006/idreamdrupal/,{'dataFile': 'sipos_2006_talanta_fig_3_KOH.csv'},{'sampleName': 'Aluminate Solution'},{'sampleName': 'Aluminate Solution'},{'sampleName': 'Potassium Hydroxide'},{'sampleName': 'Potassium Hydroxide'},{'sampleName': 'Potassium Hydroxide'},https://lampdev02.pnl.gov/bigg006/idreamdrupal/,{'dataFile': 'sipos_2006_talanta_fig_3_LiOH.csv'},{'sampleName': 'Aluminate Solution'},...,{'sampleName': 'Lithium Hydroxide'},{'sampleName': 'Lithium Hydroxide'},{'sampleName': 'Lithium Hydroxide'},https://lampdev02.pnl.gov/bigg006/idreamdrupal/,{'dataFile': 'sipos_2006_talanta_fig_3_NaOH.csv'},{'sampleName': 'Aluminate Solution'},{'sampleName': 'Aluminate Solution'},{'sampleName': 'Sodium Hydroxide'},{'sampleName': 'Sodium Hydroxide'},{'sampleName': 'Sodium Hydroxide'}
description,Extracted figures.,{'dataFile': 'sipos_2006_talanta_fig_3_KOH.csv'},{'sampleName': 'Aluminate Solution'},{'sampleName': 'Aluminate Solution'},{'sampleName': 'Potassium Hydroxide'},{'sampleName': 'Potassium Hydroxide'},{'sampleName': 'Potassium Hydroxide'},Extracted figures.,{'dataFile': 'sipos_2006_talanta_fig_3_LiOH.csv'},{'sampleName': 'Aluminate Solution'},...,{'sampleName': 'Lithium Hydroxide'},{'sampleName': 'Lithium Hydroxide'},{'sampleName': 'Lithium Hydroxide'},Extracted figures.,{'dataFile': 'sipos_2006_talanta_fig_3_NaOH.csv'},{'sampleName': 'Aluminate Solution'},{'sampleName': 'Aluminate Solution'},{'sampleName': 'Sodium Hydroxide'},{'sampleName': 'Sodium Hydroxide'},{'sampleName': 'Sodium Hydroxide'}
experimentSubType,Al_NMR,{'dataFile': 'sipos_2006_talanta_fig_3_KOH.csv'},{'sampleName': 'Aluminate Solution'},{'sampleName': 'Aluminate Solution'},{'sampleName': 'Potassium Hydroxide'},{'sampleName': 'Potassium Hydroxide'},{'sampleName': 'Potassium Hydroxide'},Al_NMR,{'dataFile': 'sipos_2006_talanta_fig_3_LiOH.csv'},{'sampleName': 'Aluminate Solution'},...,{'sampleName': 'Lithium Hydroxide'},{'sampleName': 'Lithium Hydroxide'},{'sampleName': 'Lithium Hydroxide'},Al_NMR,{'dataFile': 'sipos_2006_talanta_fig_3_NaOH.csv'},{'sampleName': 'Aluminate Solution'},{'sampleName': 'Aluminate Solution'},{'sampleName': 'Sodium Hydroxide'},{'sampleName': 'Sodium Hydroxide'},{'sampleName': 'Sodium Hydroxide'}
filename,sipos_2006_talanta_nmr_figs.json,{'dataFile': 'sipos_2006_talanta_fig_3_KOH.csv'},{'sampleName': 'Aluminate Solution'},{'sampleName': 'Aluminate Solution'},{'sampleName': 'Potassium Hydroxide'},{'sampleName': 'Potassium Hydroxide'},{'sampleName': 'Potassium Hydroxide'},sipos_2006_talanta_nmr_figs.json,{'dataFile': 'sipos_2006_talanta_fig_3_LiOH.csv'},{'sampleName': 'Aluminate Solution'},...,{'sampleName': 'Lithium Hydroxide'},{'sampleName': 'Lithium Hydroxide'},{'sampleName': 'Lithium Hydroxide'},sipos_2006_talanta_nmr_figs.json,{'dataFile': 'sipos_2006_talanta_fig_3_NaOH.csv'},{'sampleName': 'Aluminate Solution'},{'sampleName': 'Aluminate Solution'},{'sampleName': 'Sodium Hydroxide'},{'sampleName': 'Sodium Hydroxide'},{'sampleName': 'Sodium Hydroxide'}
publicReleaseDate,2006-03-13,{'dataFile': 'sipos_2006_talanta_fig_3_KOH.csv'},{'sampleName': 'Aluminate Solution'},{'sampleName': 'Aluminate Solution'},{'sampleName': 'Potassium Hydroxide'},{'sampleName': 'Potassium Hydroxide'},{'s

---